# Getting started with SCIFYsim

## Import the library

In [ ]:
import scifysim as sf
import numpy as np
import matplotlib.pyplot as plt
# optional (when using dark interfaces)
# plt.style.use("dark_background")
print("SCIFYsim version: ", sf.version)

In [ ]:
from scifysim.dummy import makesim

In [ ]:
asim = makesim("/home/romain/Documents/hi5/SCIFYsim/scifysim/config/default_R400.ini",
               target="Gl 86A")

from kernuller import pairwise_kernel
ak = pairwise_kernel(2)
myk = np.hstack((np.zeros((1,3)), ak, np.zeros((1,3))))
asim.combiner.K = myk
del ak
del myk


In [ ]:
asim.point(asim.sequence[3], asim.target)
t_exp = 3.

## Then you can start an integration

# Resolved star

* Resolved: 3 s/s
* Unresolved 1.5 s/s

In [ ]:
integ = asim.make_metrologic_exposure(asim.src.planet, asim.src.star, asim.diffuse,
                                      texp=0.1)
integ.update_enclosure(asim.lambda_science_range,
                       bottom_range=2.0e-6,
                       top_range=2.6e-6)
integ = asim.make_metrologic_exposure(asim.src.planet, asim.src.star, asim.diffuse,
                                      texp=t_exp)

integ.prepare_t_exp_base()

integ.consolidate_metrologic()

## Nice view of the results

In [ ]:
shift_step = 1/(asim.n_spec_ch+2)
outputs = np.arange(integ.summed_signal.shape[2])
isources = np.arange(len(integ.sums))
raw_sources = [integ.static[0], integ.static[1],
               integ.static[2], integ.static[3],
               integ.static, integ.starlight, integ.planetlight]
diffuse = asim.diffuse

In [ ]:
print("Number of photons for the 2 dark outputs")
for i, label in enumerate(integ.source_labels):
    #print(label + " %.1e [ph/s]"%(integ.sums[i].sum()))
    detail = integ.sums[i].sum(axis=0)[1] * integ.eta
    print(label + "    %.1e  [e-/s]"%(detail ))
for i, label in enumerate(integ.det_labels):
    #print(label + " %.1e [ph/s]"%(integ.sums[i].sum()))
    detail = integ.det_sources[i].sum(axis=0)
    print(label + "    %.1e   [e-/s]"%(detail) )

print("")
fav_output = 1
mynpix = asim.config.getfloat("spectrograph", "n_pix_split")
supsum = np.nan_to_num(np.array(integ.sums))
print(f"In total on output {fav_output}: {supsum.sum(axis=(0,1))[fav_output]:.2e} [e-/s]")
maxbin = np.max(supsum.sum(axis=0), axis=0)

# To be portable per for different resolutions:
max_density = maxbin/np.gradient(asim.lambda_science_range).mean()
print(f"In total on output {fav_output}: {max_density[fav_output]:.2e} [e-/s/m]")

fig = sf.plot_tools.plot_output_sources(asim, integ, asim.lambda_science_range, t_exp=1.)

print()
print()
print(f"\\hline")
print(f"\\hline")
print("Source & Temperature & Mean transmission & Contribution\\tablefootmark{a}\\\\ ")
print(f" & [K] & & $[e^- s^{-1}]$\\\\")
print(f"\\hline")
for i, (asource,aname) in enumerate(zip(diffuse, integ.static_list)):
    pass
    # print(f"{aname} & {asource.T:.1f} & {asource.trans(asim.lambda_science_range).mean():.2f} & {integ.sums[i].sum(axis=0)[3]:.2e} \\\\")

print(f"Enclosure & {integ.enclosure.T:.1f} & N.A. & {integ.det_sources[0].sum():.2e} \\\\")
print(f"Dark current & {60.:.1f} & N.A. & {integ.det_sources[1]*integ.det_sources[0].shape[0]:.2e} \\\\")
print(f"\\hline")

print(f"\\tablefoot{{")
print(f"\\tablefoottext{{a}}{{On one dark output, cumulated over all wavelength bands}}")
print(f"\\tablefoottext{{b}}{{For R=400}}")    
print(f"}}")
    

# Maps

## Description
The transmission maps are an important way to interface with the simulator. The maps represent the entire transmission of the instrument as a function of the relative position in the field of view. More precisely, they represent the equivalent collecting power of the whole observatory setup.

In `<simulator>.maps`, the values are stored in $m^2/sr$ representing where the solid angle prepresent the area of a pixel. It has shape $n_{chunks}, n_{chanels}, n_{outputs}, n_{pix}, n_{pix}$

For convenience, a property is available as an astropy quantity under `<simulator>.gain_map`, with units $m^2 e^-/ph$, as it includes the effect of quantum efficiency.

## Alternative backend
The maps can grow large in memory. SCIFYsim offers an alternative backend using **Dask**. [Dask](https://www.dask.org/) is a lazy and out-of-core library based of numpy. It breaks down the maps in smaller chunks and computes only at the last moment, when simplifications can be made. It can be called with `<simulator>.build_all_maps_dask`

In [ ]:
%%time
asim.build_all_maps(mapres=200, mapcrop=0.5)

In [ ]:
asim.maps.shape

In [ ]:
print(asim.gain_map.shape)
print(asim.gain_map.unit)

In [ ]:
fig = sf.plot_tools.plot_response_map(asim, sequence_index=[0],
                                outputs=np.array([3,4]),
                                dpi=200, layout="v", show=False,
                                     figsize=(3,8))
plt.tight_layout()
plt.show()

In [ ]:
asim.maps.shape

In [ ]:
my_target = "GJ 86 A"

In [ ]:
print("SCIFYsim version: ", sf.version)
sf.logit.setLevel(sf.logging.ERROR)
t_exp =1.
seed = 10
expname = "R400_base_sensitivity"
save_results = False

In [ ]:
configfile = "config/default_R400.ini"

In [ ]:
asim = sf.utilities.prepare_all(configfile, thetarget=my_target, update_params=True,
                  instrumental_errors=True, seed=seed, update_start_end=False)
asim.combiner.chromatic_matrix(asim.lambda_science_range)

In [ ]:
asim.context = sf.analysis.spectral_context(asim.config)#("config/vega_R400.ini")

In [ ]:
diffuse = [asim.src.sky, asim.src.UT, asim.src.warm_optics, asim.src.combiner, asim.src.cold_optics]


# Making some exposure

## First, point the instrument to your target

In [ ]:
asim.point(asim.sequence[3], asim.target)

## Then you can start an integration

In [ ]:
integ = asim.make_metrologic_exposure(asim.src.planet, asim.src.star, diffuse,
                                      texp=t_exp)
integ.prepare_t_exp_base()

integ.consolidate_metrologic()

## Nice view of the results

In [ ]:
shift_step = 1/(asim.n_spec_ch+2)
outputs = np.arange(integ.summed_signal.shape[2])
isources = np.arange(len(integ.sums))
raw_sources = [integ.static[0], integ.static[1],
               integ.static[2], integ.static[3],
               integ.static, integ.starlight, integ.planetlight]

In [ ]:
bottom = np.zeros_like(integ.sums[0])
pup = 1 # The pupil for which to plot the piston
print(integ.sums[0].shape)
signalplot = plt.figure(dpi=100)
bars = []
read_noise = integ.ron
for ksource, (thesource, label) in enumerate(zip(integ.sums, integ.source_labels)):
    photon_noise = np.sqrt(thesource)
    if ksource >= len(integ.static):
        inst_noise = np.std(raw_sources[ksource], axis=0)
    else:
        inst_noise = np.zeros((asim.lambda_science_range.shape[0], outputs.shape[0]))
    #print("Inst noise", ksource,  inst_noise.mean(axis=0))
    #print("Photon noise", ksource, photon_noise.mean(axis=0))
    noise = np.sqrt(photon_noise**2 + read_noise**2 + inst_noise**2)
    for ilamb in range(asim.lambda_science_range.shape[0]):
        #print(ksource, ilamb, label)
        #pdb.set_trace()
        if ilamb == 0:
            bars.append(plt.bar(outputs+shift_step*ilamb, thesource[ilamb,:], bottom=bottom[ilamb,:],
                label=label, width=shift_step, color="C%d"%ksource)) #yerr=noise[ilamb,:]
        else:
            bars.append(plt.bar(outputs+shift_step*ilamb, thesource[ilamb,:], bottom=bottom[ilamb,:],
                width=shift_step,  color="C%d"%ksource)) #yerr=noise[ilamb,:]
    bottom += thesource
#plt.legend((bars[i][0] for i in range(len(bars))), source_labels)
#Handled the legend with an condition in the loop
plt.legend(loc="upper left")
plt.xticks(outputs)
plt.xlabel(r"Output and spectral channel %.1f to %.1f $\mu m$ ($R\approx %.0f$)"%(asim.lambda_science_range[0]*1e6,
                                                                                 asim.lambda_science_range[-1]*1e6,
                                                                                 asim.R.mean()))
plt.title("Integration of %.2f s on %s"%(t_exp, asim.tarname))
plt.ylabel("Number of photons")
plt.show()

## Standard exposure


In [ ]:
dit = 1.

In [ ]:
integ = asim.make_exposure(asim.src.planet, asim.src.star, diffuse,
                                texp=dit,
                                monitor_phase=False,
                               spectro=asim.spectro)

In [ ]:
from tqdm import tqdm
n_frames = 100
mynpix = 8
diffuse = [asim.src.sky, asim.src.UT, asim.src.warm_optics, asim.src.combiner, asim.src.cold_optics]
screen_age = 0.
reveta = 1/integ.eta
full_record = True
datacube = []
dit_intensity = []
starlights = []
planetlights = []
for i in tqdm(range(n_frames)):
    if screen_age>=20. :
        print("generating screen")
        asim.injector.update_screens()
        screen_age = 0.
    integ = asim.make_exposure(asim.src.planet, asim.src.star, diffuse,
                                texp=dit,
                                monitor_phase=False,
                               spectro=None)
    datacube.append(integ.get_total(spectrograph=None,
                                    t_exp=dit,
                                    n_pixsplit=mynpix))
    dit_intensity.append(reveta * integ.forensics["Expectancy"].sum(axis=0))
    if full_record:
        starlights.append(integ.starlight.astype(np.float32))
        planetlights.append(integ.planetlight.astype(np.float32))
    integ.reset() # This can be removed after new kernel start
    screen_age += dit
datacube = np.array(datacube)
dit_intensity = np.array(dit_intensity)
starlights = np.array(starlights)
planetlights = np.array(planetlights)

## The combiner matrix

In [ ]:

plt.style.use("default")

In [ ]:
from kernuller.diagrams import plot_chromatic_matrix
fig, axs, matrix = plot_chromatic_matrix(asim.combiner.M,
                                         sf.combiners.lamb, asim.lambda_science_range,
                                         verbose=False, returnmatrix=True,minfrac=0.9,
                                         plotout=True, show=False, title="With Tepper couplers")

In [ ]:
nul_plot, cmp_plot, bar_plot, shape_plot =\
        sf.plot_tools.plot_corrector_tuning_angel_woolf(asim.corrector, asim.lambda_science_range, asim.combiner)

In [ ]:
asim.point(asim.sequence[3], asim.target)

In [ ]:
from kernuller import pairwise_kernel
ak = pairwise_kernel(2)
myk = np.hstack((np.zeros((1,3)), ak, np.zeros((1,3))))
asim.combiner.K = myk


diffobs = np.einsum("ij, mkj->mk",asim.combiner.K, dit_intensity)
diff_std = np.std(diffobs, axis=0)

In [ ]:
integ.reset()
integ = asim.make_exposure(asim.src.planet, asim.src.star, diffuse,
                                texp=dit,
                                monitor_phase=False,
                               spectro=None)
block = integ.get_total(spectrograph=None,t_exp=dit, n_pixsplit=mynpix)
print(f"datacube shape: {datacube.shape}")
print(f"dit = {dit} s")
brigh_max = np.max(np.mean(integ.forensics["Expectancy"][:,:,asim.combiner.bright], axis=0))
dark_max = np.max(np.mean(integ.forensics["Expectancy"][:,:,asim.combiner.dark], axis=0))
longest_exp_bright = 65000 / (brigh_max/dit)
longest_exp_dark = 65000 / (dark_max/dit)
print(f"Bright limit: {longest_exp_bright:.2f} s\n Dark limit: {longest_exp_dark:.2f} s")
data_std = np.std(datacube, axis=0)
diff_std = np.std(datacube[:,:,3]-datacube[:,:,4], axis=0)

integ.static = asim.computed_static
integ.mean_starlight = np.mean(starlights, axis=0)
integ.mean_planetlight = np.mean(planetlights, axis=0)
integ.mean_intensity = np.mean(dit_intensity, axis=0)

In [ ]:
mkdir /tmp/plots

In [ ]:
prof = sf.analysis.noiseprofile(integ, asim, diffobs, n_pixsplit=mynpix)
fig = prof.plot_noise_sources(asim.lambda_science_range, dit=1., show=False,
                             ymin=0.2, ymax=1.)
plt.legend(loc="upper right", fontsize="xx-small")

plt.savefig("/tmp/plots/noises.pdf", bbox_inches='tight', dpi=200)
plt.show()

# Building a correlation map

## Dealing with the noise:

This is to take into account the noise in your observation data. You should adjust it depending on your observation case

In [ ]:
from scipy.linalg import sqrtm
ndits = 100 # the number of dits taken within each chunk. (at R=400 dit~2s)
adit = 2.   # The value of dit
starmag = 4. # Magnitude of the star
             # Can also be obtained from `asim.context.get_mags_of_sim(asim)` which gives both
             # star and planet mag based on `asim.src`
amat = 1/ndits * prof.diff_noise_floor_dit(starmag, adit, matrix=True)
wmat = sqrtm(np.linalg.inv(amat))
whitenings = np.ones(len(asim.sequence))[:,None,None]*wmat[None,:,:]

## Creating some signal

Do not use this cell if you are creating your data separately.

This is a simplistic model with synthetic noise and straight signal propagation.

To be more thorough, one would use direct MC simulations.

In [ ]:
noise = np.random.multivariate_normal(mean=np.zeros(amat.shape[0]), cov=amat, size=(len(asim.sequence),1))

from lmfit import Parameters
from einops import rearrange

master_params = Parameters()
master_params.add("Sep", value = 5.) # in [mas]
master_params.add("PA", value=45.) # in [deg] East of North
master_params.add("Temperature", value=1012) # in K
master_params.add("Radius", value=0.09) # in R_sun


master_source = sf.analysis.make_source(master_params, asim.lambda_science_range, asim.src.distance)
master_light = sf.analysis.make_th_exps(asim, adit, master_source, diffuse, obs=asim.obs)
master_diff = np.einsum("k o, n w o -> n w k", asim.combiner.K, master_light)

noised_observation = master_diff + rearrange(noise, "nblock nk nwl -> nblock nwl nk")

## Creating the correlation map

In [ ]:
asim.build_all_maps_dask(mapres=100, mapcrop=0.3)
bymap = sf.utilities.extract_diffobs_map(asim.maps, asim, dit=adit,
                            postprod=None, eta=asim.integrator.eta)
#full_wmat = np.ones(bymap.shape[0])[:,None,None]*wmat[None,:,:]
cmap, xtx_map = sf.analysis.correlation_map(noised_observation, bymap.compute(),
                            postproc=whitenings,
                            K=asim.combiner.K[None,:], n_diffobs=1)
norm_map = cmap/np.sqrt(xtx_map)

## Locating the Maximum and plotting the result

Note that there is still a problem for the display of direction, as here, RA is increasing from left to right and not from right to left.

In [ ]:
cont = np.quantile(norm_map, 0.9995)
loc_norm_map = sf.utilities.get_location(norm_map, asim.map_extent, mode="cartesian")

In [ ]:
plt.figure(dpi=200)
#plt.subplot(121)
#plt.imshow(cmap, extent=asim.map_extent)
#make_cursor(loc_cmap, 3, linewidth=2.)
#plt.colorbar()
#plt.title(f"Raw map $\\mathbf{{y}}^T\mathbf{{x}}$")
#plt.subplot(122)
plt.imshow(norm_map, extent=asim.map_extent, origin="lower")
plt.colorbar()
plt.contour(norm_map, levels=[cont,], extent=asim.map_extent, origin="lower")
sf.plot_tools.make_cursor(loc_norm_map, 3, linewidth=2., flipy=False)
plt.title(f"Correlation map $\\frac{{\mathbf{{y}}^T\mathbf{{x}}}}{{\\sqrt{{\mathbf{{x}}^T\mathbf{{x}}}}}}$")
plt.xlabel("RA [mas] (reversed)")
plt.ylabel("Dec [mas]")
plt.tight_layout()
plt.show()